In [ ]:
!pip install langchain
!pip install langchain_google_genai
!pip install -U langchain-openai
!pip install langchain_community
!pip install pypdf

In [ ]:
!pip install transformers

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.chains.summarize import load_summarize_chain
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFLoader
from langchain.prompts import PromptTemplate
from langchain.text_splitter import TokenTextSplitter
from transformers import GPT2TokenizerFast


import os
#setup Langsmith monitoring
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "LangSmithEvalTest"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.langchain.plus"
os.environ["LANGCHAIN_API_KEY"] = "Your API"

#GOOGLE_API_KEY = "Your API"
openai_api_key = "Your API"

#llm = ChatGoogleGenerativeAI(model="gemini-1.5-pro-latest",google_api_key=GOOGLE_API_KEY )

from langchain_openai import ChatOpenAI
llm = ChatOpenAI(
    model="gpt-3.5-turbo",
    temperature=0.7,
    max_tokens=None,
    timeout=None,
    max_retries=6,
    verbose=True,
    api_key=openai_api_key
)

In [ ]:
#setup google driver auth from colab
from langchain.chains import load_summarize_chain
from google.colab import auth
auth.authenticate_user()

from google.colab import drive
drive.mount('/content/drive')

file_path = '/content/drive/My Drive/LLM/situationalawareness.pdf'

# Now use the text for summarization

# 1. Load PDF using a hypothetical PyPDFLoader
from langchain.document_loaders import PyPDFLoader
loader = PyPDFLoader(file_path)
documents = loader.load()

# 2a. Split the text by character

#text_splitter = RecursiveCharacterTextSplitter(chunk_size=400, chunk_overlap=20)
#all_splits = text_splitter.split_documents(documents)


# 2b. Initialize Tokenizer and TokenTextSplitter, Split by Token aware

tokenizer = GPT2TokenizerFast.from_pretrained("gpt2")
text_splitter = TokenTextSplitter(chunk_size=1000, chunk_overlap=3, encoding_name = tokenizer.name_or_path)
all_splits = text_splitter.split_documents(documents)

# 3. Prompt for better quality output

template = """

{text}
Your task is to write a concise summary using the format below.

**Key Takeaways:**
- [List the key takeaways from the paper]

**Topics Covered:**
- [List the topics discussed in the paper]

**Topic Summary:**
- [Summarize what each topic says]

{text}

**Summary:**

"""

prompt = PromptTemplate(template=template, input_variables=["text"])

chain = load_summarize_chain(llm, chain_type="stuff", prompt=prompt)

# 4. Summarize
llm = llm
chain = load_summarize_chain(llm, chain_type="stuff")

# 5. Run
summary = chain.invoke(all_splits)
print(summary)




Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


RateLimitError: Error code: 429 - {'error': {'message': 'Request too large for gpt-3.5-turbo in organization org-jwYtKPXL0e6oXGtr6QphHzN0 on tokens per min (TPM): Limit 60000, Requested 78306. The input or output tokens must be reduced in order to run successfully. Visit https://platform.openai.com/account/rate-limits to learn more.', 'type': 'tokens', 'param': None, 'code': 'rate_limit_exceeded'}}